In [2]:
tiling = [
    '........',
    '........',
    '........',
    '........',
    '........',
    '........',
    '........',
    '........',
]

In [3]:
n = len(tiling)
m = len(tiling[0])
print(n, m)

8 8


In [48]:
def count_tilings(tiling):
    if ((n + 1) * m * 2 ** m) <= 10000000:
        dp = [[[(0 if k != 0 or j != 0 or mask != 0 else 1) for mask in range(2 ** m)] for j in range(m)] for k in range(n + 1)]
    for k in range(n):
        for j in range(m):
            for mask in range(2 ** m):
                # Вертикальная доминошка
                if k < n - 1 and tiling[k][j] == '.' and tiling[k + 1][j] == '.' and (mask &  (1 << j)) == 0:
                    dp[k + ((j + 1) // m)][(j + 1) % m][mask + (1 << j)] += dp[k][j][mask]
                # Горизонтальная доминошка
                if j < m - 1 and tiling[k][j] == '.' and tiling[k][j + 1] == '.' and (mask & (3 << j)) == 0:
                    dp[k + ((j + 1) // m)][(j + 1) % m][mask + (2 << j)] += dp[k][j][mask]
                # Клетка занята
                if ((1 << j) &  mask) != 0 or tiling[k][j] != '.':
                    dp[k + ((j + 1) // m)][(j + 1) % m][(mask | (1 << j)) - (1 << j)] += dp[k][j][mask]
    return dp

In [52]:
dp = count_tilings(tiling)
print(dp[n][0][0])

12988816


In [84]:
def fill_any(tiling, dp=None):
    if dp is None:
        dp = count_tilings(tiling)
    n = len(dp) - 1
    m - len(dp[0])
    if dp[n][0][0] == 0:
        return None
    
    result = [[tiling[i][j] for j in range(m)] for i in range(n)]
    num = 0
    
    k = n
    j = 0
    mask = 0
    
    while k > 0 or j > 0:
        #print(k, j, mask)
        prev_j = j - 1
        prev_k = k
        if prev_j == -1:
            prev_j += m
            prev_k -= 1
        
        # Начинаем перебирать варианты, каким образом мы могли попасть в i, j, mask
        # Этот вариант очень не оптимален, но занимает меньше кода и все равно быстрее 
        # самого подсчета динамики
        for prev_mask in range(2 ** m):
            if prev_k < n - 1 and tiling[prev_k][prev_j] == '.' and tiling[prev_k + 1][prev_j] == '.' and \
                (prev_mask & (1 << prev_j)) == 0 and (prev_mask + (1 << prev_j)) == mask and dp[prev_k][prev_j][prev_mask] != 0:
                mask = prev_mask
                result[prev_k][prev_j] = chr(num + ord('a'))
                result[prev_k + 1][prev_j] = chr(num + ord('a'))
                #print(f'Vertical at ({prev_k}, {prev_j}) ({prev_k + 1}, {prev_j})')
                # Эвристика, чтобы переиспользовать старые буквы
                num = (num + 1) % (2 * m)
                break
            elif prev_j < m - 1 and tiling[prev_k][prev_j] == '.' and tiling[prev_k][prev_j + 1] == '.' and (prev_mask & (3 << prev_j)) == 0 and \
                prev_mask + (2 << prev_j) == mask and dp[prev_k][prev_j][prev_mask] != 0:
                mask = prev_mask
                result[prev_k][prev_j] = chr(num + ord('a'))
                result[prev_k][prev_j + 1] = chr(num + ord('a'))
                #print(f'Horisontal at ({prev_k}, {prev_j}) ({prev_k}, {prev_j + 1})')
                num = (num + 1) % (2 * m)
                break
            elif (((1 << prev_j) & prev_mask) != 0 or tiling[prev_k][prev_j] != '.') and \
                (prev_mask | (1 << prev_j)) - (1 << prev_j) == mask and dp[prev_k][prev_j][prev_mask] != 0:
                mask = prev_mask
                break
        
        j = prev_j
        k = prev_k
    
    return result

In [85]:
filling = fill_any(tiling, dp)

In [86]:
for row in filling:
    print(''.join(row))

ppoonnmm
llkkjjii
hhggffee
ddccbbaa
ppoonnmm
llkkjjii
hhggffee
ddccbbaa


In [104]:
def show_tiling(tiling, fill=True):
    if fill:
        filling = fill_any(tiling)
        if filling is not None:
            for row in filling:
                print(''.join(row))
        else:
            print('No tiling')
    else:
        for row in tiling:
            print(''.join(row))

In [88]:
tiling_no_corners_opposite = [
    '.......#',
    '........',
    '........',
    '........',
    '........',
    '........',
    '........',
    '#.......',
]
show_tiling(tiling_no_corners_opposite)

No tiling


In [89]:
tiling_no_corners_same_side = [
    '#.......',
    '........',
    '........',
    '........',
    '........',
    '........',
    '........',
    '#.......',
]
show_tiling(tiling_no_corners_same_side)

#onnmmll
kojjiihh
kgffeedd
cgbbaapp
connmmll
kojjiihh
kgffeedd
#gccbbaa


In [114]:
tiling_random = [
    '........',
    '#.#.....',
    '..#.....',
    '........',
    '........',
    '........',
    '........',
    '...#....'
]
show_tiling(tiling_random)

nnmmllkk
#j#ihhgg
fj#ieedd
fcbbaapp
ocnnmmll
okjjiihh
gkffeedd
gcc#bbaa
